<a href="https://colab.research.google.com/github/CelesteFernandezTomasincig/DataEngineering/blob/main/Api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import psycopg2
import yfinance as yf

In [11]:
#Extraccion data API
amzn = yf.Ticker('AMZN')

hist = amzn.history(period="1y")
hist['Date']=hist.index
hist=hist.reset_index(drop=True)
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
0,161.650497,162.199997,157.254501,158.755997,79056000,0.0,0.0,2022-04-06 00:00:00-04:00
1,158.399994,160.078995,154.511505,157.784500,68136000,0.0,0.0,2022-04-07 00:00:00-04:00
2,156.750000,157.368500,154.231003,154.460495,46002000,0.0,0.0,2022-04-08 00:00:00-04:00
3,152.712997,154.136505,150.534500,151.121994,52112000,0.0,0.0,2022-04-11 00:00:00-04:00
4,153.692505,155.098999,150.382996,150.787506,55178000,0.0,0.0,2022-04-12 00:00:00-04:00
...,...,...,...,...,...,...,...,...
247,102.160004,103.489998,101.949997,103.290001,56704300,0.0,0.0,2023-03-31 00:00:00-04:00
248,102.300003,103.290001,101.430000,102.410004,41135700,0.0,0.0,2023-04-03 00:00:00-04:00
249,102.750000,104.199997,102.110001,103.949997,48662500,0.0,0.0,2023-04-04 00:00:00-04:00
250,103.910004,103.910004,100.750000,101.099998,45103000,0.0,0.0,2023-04-05 00:00:00-04:00


In [12]:
#Limpieza de Data
hist= hist.rename(columns={'Open':'O', 'High':'H', 'Low':'L', 'Close':'C', 'Volume':'V','Dividens':'D', 'Stock Split':'S', 'Date':'Dat'})
hist= hist.drop(columns=['Dat'])
hist

,O,H,L,C,V,Dividends,Stock Splits
0,161.650497,162.199997,157.254501,158.755997,79056000,0.0,0.0
1,158.399994,160.078995,154.511505,157.784500,68136000,0.0,0.0
2,156.750000,157.368500,154.231003,154.460495,46002000,0.0,0.0
3,152.712997,154.136505,150.534500,151.121994,52112000,0.0,0.0
4,153.692505,155.098999,150.382996,150.787506,55178000,0.0,0.0
...,...,...,...,...,...,...,...
247,102.160004,103.489998,101.949997,103.290001,56704300,0.0,0.0
248,102.300003,103.290001,101.430000,102.410004,41135700,0.0,0.0
249,102.750000,104.199997,102.110001,103.949997,48662500,0.0,0.0
250,103.910004,103.910004,100.750000,101.099998,45103000,0.0,0.0


In [ ]:
#Creo la tabla de destino en redshift 

In [9]:
#Establecer conexion con redshift 
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="celesteft00_coderhouse"
password='B2oKky9e5cJ7'

In [16]:
#Establecer conexion con redshift 
try: 
  conn = psycopg2.connect(
      host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
      dbname='data-engineer-database',
      user='celesteft00_coderhouse',
      password='B2oKky9e5cJ7',
      port='5439'
  )
  print ("Conexion exitosa")

except Exception as e:
    print("No se pudo conectar a Redshift")
    print(e)

Conexion exitosa


In [17]:
conn

<connection object at 0x7fb9c09d1e00; dsn: 'user=celesteft00_coderhouse password=xxx dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port=5439', closed: 0>

In [18]:
#Verificar que la tabla exista
cur = conn.cursor()
cur.execute("SELECT * FROM prices")
results = cur.fetchall()
results

[]

In [19]:
#Enviar resultados
hist.head()

,O,H,L,C,V,Dividends,Stock Splits
0,161.650497,162.199997,157.254501,158.755997,79056000,0.0,0.0
1,158.399994,160.078995,154.511505,157.784500,68136000,0.0,0.0
2,156.750000,157.368500,154.231003,154.460495,46002000,0.0,0.0
3,152.712997,154.136505,150.534500,151.121994,52112000,0.0,0.0
4,153.692505,155.098999,150.382996,150.787506,55178000,0.0,0.0


In [23]:
from psycopg2.extras import execute_values
cur = conn.cursor()
table_name ="prices"
columns = ['O', 'H', 'L', 'C', 'V', 'D', 'S']
values = [tuple(x) for x in hist.to_numpy()]
insert_sql = f"INSERT INTO {table_name}({', '.join(columns)}) VALUES %s"
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute('COMMIT')

In [24]:
cur.close()
conn.close()
